In [1]:
import pandas as pd
import numpy as np
import os
import matplotlib.pyplot as plt
from datetime import datetime
from sklearn.preprocessing import MinMaxScaler, RobustScaler, StandardScaler
import time

import tensorflow as tf
tf.random.set_seed(42)

def mape(y_true, y_pred):
    y_true, y_pred = np.array(y_true), np.array(y_pred)
    return np.mean(np.abs((y_true - y_pred) / max(y_true))) * 100

def RMSELoss(yhat,y):
    return torch.sqrt(torch.mean((yhat-y)**2))

print("Num GPUs Available: ", len(tf.config.list_physical_devices('GPU')))

Num GPUs Available:  1


In [2]:
# Specifying the path of file
path= r"C:\Users\sashah8\OneDrive - North Carolina State University\Desktop\Q4\Data\L4\data_cleaned"

# Listing just the required files
fileList = os.listdir(path)
fileList.remove("Date.csv")

# Previewing the required file names
print(fileList)

['Capital.csv', 'Central.csv', 'Dunwoody.csv', 'Genese.csv', 'Hudson.csv', 'LongIsland.csv', 'Milwood.csv', 'NYC.csv']


In [3]:
# Just for previewing the columns
pd.read_csv(os.path.join(path, fileList[4]))

,Demand,Normalized_net,Temprature
0,1069.8,0.491246,-5.8
1,1020.4,0.468561,-6.1
2,982.8,0.451296,-7.5
3,962.1,0.441790,-8.4
4,957.5,0.439678,-7.4
...,...,...,...
43819,1180.2,0.541941,3.4
43820,1134.0,0.520726,3.3
43821,1091.2,0.501072,2.6
43822,1041.3,0.478159,2.7


### PROCESSING THE DATA

In [4]:
# Restricting the load data to a single zone
fileName  = fileList[4]

zoneData = pd.read_csv(os.path.join(path, fileName), usecols= ["Demand", "Temprature"])
zoneData.columns = ["Load_"+str(fileName[:-4]), "Temp_"+str(fileName[:-4])]

# For fixing one missing row
# zoneData = pd.concat([zoneData.loc[[0]], zoneData], axis= 0).reset_index(drop= True)

zoneData

,Load_Hudson,Temp_Hudson
0,1069.8,-5.8
1,1020.4,-6.1
2,982.8,-7.5
3,962.1,-8.4
4,957.5,-7.4
...,...,...
43819,1180.2,3.4
43820,1134.0,3.3
43821,1091.2,2.6
43822,1041.3,2.7


### FEATURE ENGINEERING AND SPLITTING INTO TRAIN-TEST SPLIT

In [5]:
def lagGenerator(dF, lagList):
    dF_temp = dF.filter(regex= "Load").copy()
    
    for i in lagList:
        dF_temp.loc[:, "Lag-" + str(i)] = dF_temp.iloc[:,[0]].shift(i*24)
    
    return(dF_temp.filter(regex= "Lag"))

In [6]:
Load_lags = lagGenerator(dF=zoneData, lagList=[2,7])
Load_lags.head(2)

,Lag-2,Lag-7
0,NaN,NaN
1,NaN,NaN


In [7]:
dataDf = pd.concat([zoneData, Load_lags], axis= 1).dropna(axis= 0).reset_index(drop= True)
dataDf

,Load_Hudson,Temp_Hudson,Lag-2,Lag-7
0,1177.7,-13.9,1057.1,1069.8
1,1140.2,-14.2,1026.3,1020.4
2,1122.8,-14.8,1009.9,982.8
3,1119.9,-15.6,1004.1,962.1
4,1138.0,-16.0,1023.2,957.5
...,...,...,...,...
43651,1180.2,3.4,1222.0,1172.9
43652,1134.0,3.3,1186.6,1151.3
43653,1091.2,2.6,1132.2,1130.6
43654,1041.3,2.7,1062.8,1094.8


In [8]:
# Setting the train-test split ratio
ratio = 0.1

In [9]:
Num_train, Num_test = int(len(dataDf)/24 * (1 - ratio)) * 24, int(len(dataDf)/24 * ratio) * 24

In [10]:
# Splitting the data into train and test
data_train, data_test = dataDf[:Num_train], dataDf[Num_train:]

In [11]:
Num_train, Num_valid = int(len(data_train)/24 * (1 - ratio)) * 24, int(len(data_train)/24 * ratio) * 24

In [12]:
# Splitting the data into train and test
data_train, data_valid = data_train[:Num_train], data_train[Num_train:]

### SPLITTING INTO X AND Y PART WITH SEQUENCE BATCHING

In [13]:
X_train_df, Y_train_df = data_train.iloc[:, 1:], data_train.iloc[:, :1]

X_valid_df, Y_valid_df = data_valid.iloc[:, 1:], data_valid.iloc[:, :1]

X_test_df, Y_test_df = data_test.iloc[:, 1:], data_test.iloc[:, :1]

In [14]:
# Preparing the scaler
ValScaler = MinMaxScaler()

X_train = ValScaler.fit_transform(X_train_df)
X_valid = ValScaler.transform(X_valid_df)
X_test = ValScaler.transform(X_test_df)

Y_train = ValScaler.fit_transform(Y_train_df)
Y_valid = ValScaler.transform(Y_valid_df)
Y_test = ValScaler.transform(Y_test_df)

In [15]:
def splitData(dataset):
    data = np.array(np.split(dataset, len(dataset)/24))
    return data

In [16]:
def supervisedSet(X_part, Y_part, steps= 24):
    X_array, Y_array = np.array(X_part), np.array(Y_part)
    X, Y = [], []
    
    for i in range(len(X_array)):
        endIdx = i + steps
        
        if endIdx > len(X_array):
            break
        
        batch_X, batch_Y = X_array[i:endIdx], Y_array[i:endIdx]
        
        X.append(batch_X)
        Y.append(batch_Y)
        
    return np.array(X), np.array(Y)

In [17]:
X_train, Y_train = supervisedSet(X_train, Y_train)

X_valid, Y_valid = splitData(X_valid), splitData(Y_valid)

X_test, Y_test = splitData(X_test), splitData(Y_test)

### WORKING ON THE MODEL

In [18]:
epochs = 100
batchSize = 32

In [19]:
# Encoder-decoder architecture [tanh required to use tensorflow on gpu]
model = tf.keras.Sequential([
    tf.keras.layers.LSTM(units= 128, activation= "tanh", input_shape= (X_train.shape[1], X_train.shape[2]), dropout= 0.2),
    tf.keras.layers.RepeatVector(Y_train.shape[1]),
    tf.keras.layers.LSTM(units= 128, activation= "tanh", return_sequences= True, dropout= 0.2),
    tf.keras.layers.TimeDistributed(tf.keras.layers.Dense(units= 128, activation= "relu")),
    tf.keras.layers.Dropout(0.2),
    tf.keras.layers.TimeDistributed(tf.keras.layers.Dense(units= 1))
])

model.compile(loss= "mse", optimizer= "adam",  metrics= ['mape', 'mae'])

In [20]:
# model = tf.keras.Sequential([
#     tf.keras.layers.LSTM(units= 256, activation= "tanh", input_shape= (X_train.shape[1], X_train.shape[2])),
#     tf.keras.layers.Dense(units= 128, activation= "tanh"),
#     tf.keras.layers.Dense(units= 1)
# ])

# model.compile(loss= "mse", optimizer= "adam",  metrics= ['mape', 'mae'])

In [21]:
# Storing the start time of model fitting process
startTime = time.time()

In [22]:
model.fit(X_train, Y_train, epochs= epochs, batch_size= batchSize, verbose= 1, shuffle= True, validation_data= (X_valid, Y_valid))

Epoch 1/100
1105/1105 [==============================] - 17s 11ms/step - loss: 0.0114 - mape: 5708.0493 - mae: 0.0812 - val_loss: 0.0078 - val_mape: 83.9004 - val_mae: 0.0721
Epoch 2/100
1105/1105 [==============================] - 12s 11ms/step - loss: 0.0063 - mape: 5086.7212 - mae: 0.0609 - val_loss: 0.0091 - val_mape: 86.6254 - val_mae: 0.0747
Epoch 3/100
1105/1105 [==============================] - 12s 11ms/step - loss: 0.0042 - mape: 4666.2681 - mae: 0.0500 - val_loss: 0.0077 - val_mape: 73.8086 - val_mae: 0.0640
Epoch 4/100
1105/1105 [==============================] - 12s 11ms/step - loss: 0.0038 - mape: 4825.4062 - mae: 0.0472 - val_loss: 0.0070 - val_mape: 67.9959 - val_mae: 0.0605
Epoch 5/100
1105/1105 [==============================] - 12s 11ms/step - loss: 0.0034 - mape: 4827.9199 - mae: 0.0447 - val_loss: 0.0080 - val_mape: 48.1421 - val_mae: 0.0630
Epoch 6/100
1105/1105 [==============================] - 12s 11ms/step - loss: 0.0032 - mape: 4629.1265 - mae: 0.0430 - val_l

1105/1105 [==============================] - 12s 11ms/step - loss: 0.0017 - mape: 2242.4067 - mae: 0.0307 - val_loss: 0.0067 - val_mape: 61.6317 - val_mae: 0.0563
Epoch 48/100
1105/1105 [==============================] - 12s 11ms/step - loss: 0.0017 - mape: 2243.5627 - mae: 0.0308 - val_loss: 0.0069 - val_mape: 56.1892 - val_mae: 0.0576
Epoch 49/100
1105/1105 [==============================] - 12s 11ms/step - loss: 0.0017 - mape: 2218.6287 - mae: 0.0303 - val_loss: 0.0073 - val_mape: 53.8080 - val_mae: 0.0587
Epoch 50/100
1105/1105 [==============================] - 12s 11ms/step - loss: 0.0017 - mape: 1984.2080 - mae: 0.0303 - val_loss: 0.0067 - val_mape: 49.6821 - val_mae: 0.0547
Epoch 51/100
1105/1105 [==============================] - 12s 11ms/step - loss: 0.0016 - mape: 2073.9465 - mae: 0.0301 - val_loss: 0.0074 - val_mape: 48.4523 - val_mae: 0.0589
Epoch 52/100
1105/1105 [==============================] - 12s 11ms/step - loss: 0.0016 - mape: 2291.3325 - mae: 0.0301 - val_loss: 0.

1105/1105 [==============================] - 12s 11ms/step - loss: 0.0010 - mape: 1393.4155 - mae: 0.0238 - val_loss: 0.0074 - val_mape: 59.6160 - val_mae: 0.0588
Epoch 94/100
1105/1105 [==============================] - 12s 11ms/step - loss: 9.7973e-04 - mape: 846.7833 - mae: 0.0235 - val_loss: 0.0072 - val_mape: 62.0860 - val_mae: 0.0588
Epoch 95/100
1105/1105 [==============================] - 12s 11ms/step - loss: 9.7492e-04 - mape: 883.6955 - mae: 0.0234 - val_loss: 0.0068 - val_mape: 60.8730 - val_mae: 0.0563
Epoch 96/100
1105/1105 [==============================] - 12s 11ms/step - loss: 9.6532e-04 - mape: 1324.6558 - mae: 0.0233 - val_loss: 0.0070 - val_mape: 64.3646 - val_mae: 0.0572
Epoch 97/100
1105/1105 [==============================] - 12s 11ms/step - loss: 9.5864e-04 - mape: 1169.1884 - mae: 0.0232 - val_loss: 0.0078 - val_mape: 66.6075 - val_mae: 0.0616
Epoch 98/100
1105/1105 [==============================] - 12s 11ms/step - loss: 9.3244e-04 - mape: 997.8507 - mae: 0.02

In [23]:
calcExecTime = (time.time() - startTime)
print('Total execution time in seconds: ' + str(calcExecTime))

Total execution time in seconds: 1205.4449772834778


### MAKING PREDICTIONS AND GENERATING RESULTS

In [24]:
predictions = model.predict(X_test)

6/6 [==============================] - 1s 6ms/step


In [25]:
predArr = predictions.reshape(predictions.shape[0] * predictions.shape[1], predictions.shape[2]).ravel()
predArr

array([0.16865605, 0.14181146, 0.13019302, ..., 0.38698304, 0.3483673 ,
       0.3154336 ], dtype=float32)

In [26]:
Y_testArr = Y_test.reshape(Y_test.shape[0] * Y_test.shape[1], Y_test.shape[2]).ravel()
Y_testArr

array([0.31748091, 0.28677933, 0.2639968 , ..., 0.33813138, 0.30773438,
       0.27910393])

In [27]:
def inverse_transform(df):
    column_names = df.columns
    for col in column_names:
        df[col] = ValScaler.inverse_transform(df[[col]]) 
    return df

In [28]:
comparisionDf = pd.DataFrame(data= {"Values": Y_testArr, "Predictions": predArr})
comparisionDf = inverse_transform(comparisionDf)
comparisionDf

,Values,Predictions
0,1057.3,812.987732
1,1006.9,768.919434
2,969.5,749.846497
3,953.6,756.933594
4,979.1,796.581848
...,...,...
4363,1180.2,1277.204712
4364,1134.0,1233.523682
4365,1091.2,1171.395386
4366,1041.3,1108.003418


In [29]:
mape(comparisionDf["Values"], comparisionDf["Predictions"])

7.798578058457157